# EventTest

In [2]:
%%writefile ../src/EventTest.sol
pragma solidity ^0.6.0;
contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Overwriting ../src/EventTest.sol


In [3]:
!solc ../src/EventTest.sol --combined-json abi > ..\src\EventTestABI.json

In [4]:
!solc ../src/EventTest.sol --combined-json bin > ..\src\EventTestBIN.json

## 배포

In [9]:
%%writefile ../src/EventTest.js
var Web3=require('web3');
var _abiJson = require('./EventTestABI.json');
var _binJson = require('./EventTestBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); // reading ['src//FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin=_binJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/EventTest.js


In [42]:
!node ../src/EventTest.js

- contract name:  ../src/EventTest.sol:EventTest
Deploying the contract from 0x84667E9479070da2582A13Fb06494BC3B919edCC
hash: 0x4cf517531e406734c1715b19f1f4e1b8d5cebc48881803274fa0e56ebfe3f8fc
---> The contract deployed to: 0x0c2D722b03ABFc82c7aF7844dB021C19E64a4f93


#### gas 사용량, contractAddress
```!geth --exec 'eth.getTransactionReceipt("트랜잭션 해시");'
attach http://localhost:8345```

## 사용

#### HttpProvider 사용 (event가 점화되지 않음)

In [20]:
%%writefile ../src/EventTestUse.js
var Web3=require('web3');
var _abiJson = require('./EventTestABI.json');
var _binJson = require('./EventTestBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _test = new web3.eth.Contract(_abiArray, '0x122c8Ad0Dde32B4Ab9B0fd4153Cd65dDc5705A8c');

var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
   
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting ../src/EventTestUse.js


In [21]:
!node ../src/EventTestUse.js

- contract name:  ../src/EventTest.sol:EventTest
Balance before: 99999900779000000000
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99999877967000000000
Balance diff: 22812000010240


#### WebsocketProvider 사용

In [38]:
%%writefile ../src/EventTestUse.js
var Web3=require('web3');
var _abiJson = require('./EventTestABI.json');
var _binJson = require('./EventTestBIN.json');

var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _test = new web3.eth.Contract(_abiArray, '0xfa452f66Bc712810F478d4697B9a3FB8e9B1ac9A');

var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        
        //대기 후 종료
        setTimeout(function() {
          process.exit(1);
        }, 1000);
    }
});


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
   
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting ../src/EventTestUse.js


In [39]:
!node ../src/EventTestUse.js

- contract name:  ../src/EventTest.sol:EventTest
Event fired: {"0":"Hello World!","my":"Hello World!"}
Balance before: 99999855155000000000
Event fired: {"0":"Hello World!","my":"Hello World!"}
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99999832343000000000
Balance diff: 22811999993856


### 파일에 로그 쓰기

In [75]:
%%writefile ../src/EventTestUse.js
var Web3=require('web3');
var fs=require('fs');
var _abiJson = require('./EventTestABI.json');
var _binJson = require('./EventTestBIN.json');

var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);

var _test = new web3.eth.Contract(_abiArray, '0x0c2D722b03ABFc82c7aF7844dB021C19E64a4f93');

var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        log = JSON.stringify(result.returnValues);
        console.log("Event fired: " + log);
        
        // 로그를 파일로 작성. writeFile은 파일이 기존에 있어야하고, 이를 덮어씀
        // appendFile은 기존에 없어도 새로 생성을 하고, 기존의 내용에 이어서 씀
        fs.writeFile("../src/EventTestLog.txt", log, "utf-8", function(e) {
            if(!e) {
                console.log(">> Writing to file");
            }
        });
                    
        //대기 후 종료
        setTimeout(function() {
          process.exit(1);
        }, 1000);
    }
});


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
   
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting ../src/EventTestUse.js


In [76]:
!node ../src/EventTestUse.js

- contract name:  ../src/EventTest.sol:EventTest
Balance before: 99999718283000000000
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99999695471000000000
Balance diff: 22812000010240


In [77]:
!type ..\src\EventTestLog.txt

{"0":"Hello World!","my":"Hello World!"}
